# 작업 2유형
- https://www.datamanim.com/dataset/03_dataq/typetwo.html#id3

# 1. 서비스 이탈예측 데이터(Classification)

> Attention
- 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
- 데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
- X_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
- X_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
- y_test(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv

#### 0. 시험 환경 세팅

In [ ]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv")

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6499, 12), (6499, 2), (3501, 12), (3501, 2))

#### 1. 라이브러리 및 데이터 호출

In [ ]:
# 필요한 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd

- CustomerId 컬럼 제거
- y_train의 Exited 컬럼을 label data 로 지정

In [ ]:
del X_train["CustomerId"]
X_train.head()

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


In [ ]:
target = y_train["Exited"]

#### 2. EDA
- X_train 의 기초 통계량, null 값 확인

In [ ]:
# X_train 의 기초 통계량, null 값 확인

In [ ]:
#기초통계량
X_train.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000
mean,650.396830,38.957070,5.041545,76836.581068,1.519772,0.708878,0.514387,100346.564524
std,96.618957,10.502803,2.891779,62407.570894,0.578975,0.454314,0.499831,57944.655305
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50907.565000
50%,651.000000,37.000000,5.000000,97560.160000,1.000000,1.000000,1.000000,100496.840000
75%,718.000000,44.000000,8.000000,127844.690000,2.000000,1.000000,1.000000,150480.155000
max,850.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199970.740000


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Surname          6499 non-null   object 
 1   CreditScore      6499 non-null   int64  
 2   Geography        6499 non-null   object 
 3   Gender           6499 non-null   object 
 4   Age              6499 non-null   int64  
 5   Tenure           6499 non-null   int64  
 6   Balance          6499 non-null   float64
 7   NumOfProducts    6499 non-null   int64  
 8   HasCrCard        6499 non-null   int64  
 9   IsActiveMember   6499 non-null   int64  
 10  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(6), object(3)
memory usage: 558.6+ KB


#### 3. Preprocessing
- Gender : 띄어쓰기 제거 및 대문자 변경
- 수치형 컬럼 : 정규화(MinMaxScaler)
- 범주형 컬럼 : 인코딩(LabelEncoder)

In [ ]:
# 띄어쓰기 제거
# 대문자 로 전부 변경

In [ ]:
X_train["Gender"] = X_train["Gender"].str.upper()
X_train["Gender"]

0       FEMALE
1         MALE
2       FEMALE
3       FEMALE
4       FEMALE
         ...  
6494      MALE
6495      MALE
6496    FEMALE
6497    FEMALE
6498    FEMALE
Name: Gender, Length: 6499, dtype: object

In [ ]:
X_train["Gender"] = X_train["Gender"].str.strip()
X_train["Gender"].unique()

array(['FEMALE', 'MALE'], dtype=object)

In [ ]:
# 범주형 컬럼 인코딩

In [ ]:
# object column labeling
from sklearn.preprocessing import LabelEncoder
obj_col = X_train.select_dtypes(include='object').columns
le = [LabelEncoder() for i in range(len(obj_col))]
for i in range(len(obj_col)):
    X_train[obj_col[i]] = le[i].fit_transform(X_train[obj_col[i]])

In [ ]:
X_train

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,2274,791,1,0,35,7,52436.20,1,1,0,161051.75
1,194,705,1,1,42,8,166685.92,2,1,1,55313.51
2,958,543,0,0,31,4,138317.94,1,0,0,61843.73
3,1728,709,0,0,32,2,0.00,2,0,0,109681.29
4,2067,714,1,0,36,1,101609.01,2,1,1,447.73
...,...,...,...,...,...,...,...,...,...,...,...
6494,1315,696,2,1,24,9,0.00,1,0,0,10883.52
6495,2139,513,1,1,34,7,60515.13,1,0,0,124571.09
6496,1467,663,2,0,22,9,0.00,1,1,0,29135.89
6497,965,635,2,0,48,2,0.00,2,1,1,136551.25


In [ ]:
# 수치형 컬럼 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_sc

array([[0.99388112, 0.882     , 0.5       , ..., 1.        , 0.        ,
        0.80536531],
       [0.08479021, 0.71      , 0.5       , ..., 1.        , 1.        ,
        0.27656612],
       [0.41870629, 0.386     , 0.        , ..., 0.        , 0.        ,
        0.30922389],
       ...,
       [0.64117133, 0.626     , 1.        , ..., 1.        , 0.        ,
        0.14565129],
       [0.42176573, 0.57      , 1.        , ..., 1.        , 1.        ,
        0.68283779],
       [0.97683566, 0.494     , 0.        , ..., 1.        , 1.        ,
        0.03572985]])

#### 4. Modeling
- 분류MODEL 을 이용해 서비스 이탈예측
- 각 MODEL 의 기본값을 이용해 예측해 보세요
    - KNN
    - SVC
    - RandomForestClassifier
    - XGBClassifier
- accuracy_score, roc_auc_score 를 이용해 평가
    - accuracy : 0.85
    - auc : 0.85
- gridsearch를 활용한 best model 과 best_parameter, RandomForestClassifier의 feature_importances_ 는 어떤값을 나타 내는가?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVR, SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier